In [1]:
# pip install openai wikipedia

In [2]:
import openai
import os
import wikipedia

# 1.) Set up OpenAI and the enviornment


In [3]:
apikey = "Purchase your own apikey"

In [4]:
openai.api_key = apikey

In [5]:
client = openai.OpenAI(
    api_key = openai.api_key
)

# 2.) Use the wikipedia api to get a function that pulls in the text of a wikipedia page

In [6]:
dir(wikipedia)

['API_URL',
 'BeautifulSoup',
 'Decimal',
 'DisambiguationError',
 'HTTPTimeoutError',
 'ODD_ERROR_MESSAGE',
 'PageError',
 'RATE_LIMIT',
 'RATE_LIMIT_LAST_CALL',
 'RATE_LIMIT_MIN_WAIT',
 'RedirectError',
 'USER_AGENT',
 'WikipediaException',
 'WikipediaPage',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'cache',
 'datetime',
 'debug',
 'donate',
 'exceptions',
 'geosearch',
 'languages',
 'page',
 'random',
 're',
 'requests',
 'search',
 'set_lang',
 'set_rate_limiting',
 'set_user_agent',
 'stdout_encode',
 'suggest',
 'summary',
 'sys',
 'time',
 'timedelta',
 'unicode_literals',
 'util',
 'wikipedia']

In [7]:
page_titles = ["Artificial Intelligence", "Machine Learning"]

In [8]:
page_titles = page_titles[0]

In [9]:
search_results = wikipedia.search(page_titles)
search_results

['Artificial intelligence',
 'Generative artificial intelligence',
 'Artificial general intelligence',
 'A.I. Artificial Intelligence',
 'Applications of artificial intelligence',
 'Hallucination (artificial intelligence)',
 'Artificial intelligence art',
 'Ethics of artificial intelligence',
 'History of artificial intelligence',
 'Artificial intelligence in healthcare']

In [10]:
page = wikipedia.page(search_results[0])

In [11]:
dir(page)

['_WikipediaPage__continued_query',
 '_WikipediaPage__load',
 '_WikipediaPage__title_query_param',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'categories',
 'content',
 'coordinates',
 'html',
 'images',
 'links',
 'original_title',
 'pageid',
 'parent_id',
 'references',
 'revision_id',
 'section',
 'sections',
 'summary',
 'title',
 'url']

In [12]:
page.content

'Artificial intelligence (AI) is the intelligence of machines or software, as opposed to the intelligence of living beings, primarily of humans. It is a field of study in computer science that develops and studies intelligent machines. Such machines may be called AIs.\nAI technology is widely used throughout industry, government, and science. Some high-profile applications are: advanced web search engines (e.g., Google Search), recommendation systems (used by YouTube, Amazon, and Netflix), interacting via human speech (such as Google Assistant, Siri, and Alexa), self-driving cars (e.g., Waymo), generative and creative tools (ChatGPT and AI art), and superhuman play and analysis in strategy games (such as chess and Go).Alan Turing was the first person to conduct substantial research in the field that he called machine intelligence. Artificial intelligence was founded as an academic discipline in 1956. The field went through multiple cycles of optimism, followed by periods of disappointm

In [13]:
def get_wikipedia_content(page_titles):
    search_results = wikipedia.search(page_titles)
    page = wikipedia.page(search_results[0])
    return(page.content)

# 3.) Build a chatgpt bot that will analyze the text given and try to locate any false info

In [14]:
wiki_content = get_wikipedia_content(page_titles)

chat_completion = client.chat.completions.create(
    model = "gpt-4",
    messages = [{"role": "system", "content": "You are a summary assistant at Wikipedia, I will pass you an article and please tell me if any of the information is false."},
                {"role": "user", "content": wiki_content[:8192]}]
)

In [15]:
chat_completion = client.chat.completions.create(
    model = "gpt-4",
    messages = [{"role": "system", "content": "I will be giving you an article. I am looking for false information. I want to capture all false information if there is even a small potential for it to be wrong, please return it. Please concisely list only the false information found."},
                {"role": "user", "content": wiki_content[:8192]}]
)

In [16]:
print(chat_completion.choices[0].message.content)

1. The statement "Alan Turing was the first person to conduct substantial research in the field that he called machine intelligence." is false. While Alan Turing made significant contributions to the field, research in machine intelligence wasn't uniquely credited to him.
2. The article mentions that "AI also draws upon psychology, linguistics, philosophy, neuroscience, and other fields." It's incorrect to assume that all AI research draws upon these fields; it depends on the specific area of AI research.
3. The claim that the field of AI research includes the goal of 'support for robotics' is misleading. While AI is used in robotics, the field of AI research doesn't necessarily include support for robotics as one of its traditional goals.
4. The claim "A Markov decision process has a transition model that describes the probability that a particular action will change the state in a particular way, and a reward function that supplies the utility of each state and the cost of each actio

In [17]:
def chatgpt_error_correction(text):
    chat_completion = client.chat.completions.create(
        model = "gpt-4",
        messages = [{"role": "system", "content": "I will be giving you an article. I am looking for false information. I want to capture all false information if there is even a small potential for it to be wrong, please return it. Please concisely list only the false information found."},
                    {"role": "user", "content": text[:8192]}]
)
    return print(chat_completion.choices[0].message.content)

# 4.) Make a for loop and check a few wikipedia pages and return a report of any potentially false info via wikipedia

In [18]:
page_titles = ["Artificial Intelligence", "UCLA"]

In [19]:
for page_title in page_titles:
    print("___________" + page_title)
    try:
        text = get_wikipedia_content(page_title)
        print(chatgpt_error_correction(text[:8192]))
    except:
        print("ERROR")

___________Artificial Intelligence
1. "Alan Turing was the first person to conduct substantial research in the field that he called machine intelligence." - While Alan Turing was a pioneer in the field, he was not the first person to conduct substantial research as there were several others who contributed as well.

2. "Artificial intelligence was founded as an academic discipline in 1956." - This can be considered as incorrect because though the term 'Artificial Intelligence' was coined at the Dartmouth conference in 1956, the study of AI as a discipline has roots going back much earlier.

3. "Funding and interest vastly increased after 2012 when deep learning surpassed all previous AI techniques, and after 2017 with the transformer architecture." - This statement inaccurately portrays deep learning as surpassing all previous AI techniques after 2012. While deep learning significantly advanced the field, other techniques also continued to be developed and used.

4. "Even humans rarely